In [1]:
#%%Forcolabonly
from google.colab import drive
drive.mount('/content/gdrive')
#%%

Mounted at /content/gdrive


In [3]:
!pip install datasets
!pip install transformers
!pip install torchcontrib
import torch
from torch import nn
import torch.nn.functional as F
from torch import BoolTensor, FloatTensor, LongTensor, ByteTensor
from typing import List, Optional, Sequence, Union, Callable, Dict, Any, Tuple
from datasets import load_dataset
import transformers
from transformers import DistilBertTokenizerFast, BertPreTrainedModel, get_linear_schedule_with_warmup, AdamW
from torchcontrib.optim import SWA
import regex as re
import numpy as np
from tqdm import tqdm

     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 245kB 7.5MB/s 
     |████████████████████████████████| 17.7MB 1.3MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 1.5MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 21.1MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=97aa639cf7a504721177ad7dc9850e988f797c72b2a3014c18be81905a176f23
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-cp36-none-any.whl size=7531 sha256=bca5e45d429ad99b6cb2c025341d795b9515745dd2e693cc05615a5ceca5c037
  Stored in directory: /root/.cache/pip/wheels/06/06/7b/a5f5920bbf4f12a2c927e438fa

In [102]:
tags=sorted(list('.?!,;:-—…'))
tag2id = {tag: id+1 for id, tag in enumerate(tags)}
tag2id[' ']=0
tag2id['']=-100
id2tag = {id: tag for tag, id in tag2id.items()}
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
class PunctuationDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids:LongTensor, attention_mask:FloatTensor, labels:Optional[LongTensor] = None) -> None:
        """
        :param input_ids: tokenids
        :param attention_mask: attention_mask, null->0
        :param labels: true labels, optional
        :return None
        """
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __getitem__(self, idx):
        """:param idx: implement index"""
        return {'input_ids': torch.as_tensor(self.input_ids[idx],dtype=torch.long),
                'attention_mask': torch.as_tensor(self.attention_mask[idx],dtype=torch.uint8),
                'labels': torch.as_tensor(self.labels[idx],dtype=torch.long)}

    def view(self,idx:int)->str:
        """:param idx(int): returns readable format of single input_ids and labels in the form of readable text"""
        return ' '.join([''.join(x) for x in list(zip(tokenizer.convert_ids_to_tokens(self.input_ids[idx]),[id2tag[x] for x in self.labels[idx].tolist()]))])

    def __len__(self)->int:
        return len(self.labels)

In [284]:
class config:
  def __init__(self):
    self.max_len=128
    self.overlap = 126
    self.train_batch_size = 4
    self.dev_batch_size = 4
    self.gpu_device = 'cpu' # 'cuda:0' #
    self.freeze_epochs = 20
    self.freeze_lr = 1e-4
    self.unfreeze_epochs = 20
    self.unfreeze_layers = 6
    self.unfreeze_lr = 1e-5
    self.base_model_path = 'distilbert-base-uncased'
    self.train_dataset = '/content/gdrive/MyDrive/ASR/ted_talks_processed.train.pt'
    self.dev_dataset = '/content/gdrive/MyDrive/ASR/ted_talks_processed.dev.pt'
    self.alpha = 0.8
    self.hidden_dropout_prob = 0.3
    self.embedding_dim = 768
    self.num_labels = 10
    self.hidden_dim = 128
    self.self_adjusting = True
    self.square_denominator = False
    self.use_crf = False
    self.use_dice = False
    self.reduction='mean'
    self.model_name = 'bertcrf'
    self.model_path = "/content/gdrive/MyDrive/ASR/logs/models/"
    self.log_dir='/tmp/tensorboard_logs'
config = transformers.configuration_utils.PretrainedConfig.from_dict(config().__dict__)

In [157]:
device = torch.device(config.gpu_device) if torch.cuda.is_available() else torch.device('cpu')
train_dataset=PunctuationDataset(**torch.load(config.train_dataset,map_location=device)[:])
dev_dataset=PunctuationDataset(**torch.load(config.dev_dataset,map_location=device)[:])
train_dataset.view(-1000)

"[CLS]  turns  activism  into  terrorism  if  it  causes  a  loss  of  profits. now  most  people  never  even  heard  about  this  law, including  members  of  congress. less  than  one  percent  were  in  the  room  when  it  passed  the  house. the  rest  were  outside  at  a  new  memorial. they  were  praising  dr. king  as  his  style  of  activism  was  branded  as  terrorism  if  done  in  the  name  of  animals  or  the  environment. supporters  say  laws  like  this  are  needed  for  the  ex  ##tre  ##mist  ##s: the  van  ##dal  ##s, the  arson  ##ists, the  radicals. but  right  now, companies  like  trans  ##cana  ##da  are  briefing  police  in  presentations  like  this  one  about  how  to  prose  ##cute  non  ##vio  ##lent  protesters  as  terrorists. the  fbi  '  s  training  documents  on  eco- [SEP] "

In [159]:
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=config.train_batch_size, num_workers=4)
dev_dataloader=torch.utils.data.DataLoader(dev_dataset, batch_size=config.dev_batch_size, num_workers=2)
{x:y.shape for x,y in next(iter(train_dataloader)).items()},{x:y.shape for x,y in next(iter(dev_dataloader)).items()} #(batch_size, seq_len)

({'attention_mask': torch.Size([4, 128]),
  'input_ids': torch.Size([4, 128]),
  'labels': torch.Size([4, 128])},
 {'attention_mask': torch.Size([4, 128]),
  'input_ids': torch.Size([4, 128]),
  'labels': torch.Size([4, 128])})

In [233]:
class DiceLoss(nn.Module):
    r"""
    Creates a criterion that optimizes a multi-class Self-adjusting Dice Loss
    ("Dice Loss for Data-imbalanced NLP Tasks" paper)
    Args:
        alpha (float): a factor to push down the weight of easy examples
        gamma (float): a factor added to both the nominator and the denominator for smoothing purposes
    """
    def __init__(self,
                 smooth: Optional[float] = 1e-8,
                #  square_denominator: Optional[bool] = False,
                 self_adjusting: Optional[bool] = False,
                 reduction: Optional[str] = "mean",
                 alpha: float = 1.0,
                 num_classes: int = 10,
                 weight=1, #int or list
                 ) -> None:
        super(DiceLoss, self).__init__()
        # self.ignore_index = ignore_index
        self.reduction = reduction
        self.self_adjusting = self_adjusting
        self.num_classes = num_classes
        self.alpha = alpha
        self.smooth = smooth
        # self.square_denominator = square_denominator
        self.weight=weight
    def forward(self,
                pred: FloatTensor,
                target: LongTensor,
                mask: ByteTensor,
                ) -> torch.Tensor:
        pred_soft = torch.softmax(pred,-1) #(batch_size,seq_len,num_labels)->(batch_size,seq_len,num_labels), sum along num_labels to 1
        target_one_hot=F.one_hot(target,num_classes=self.num_classes) #(b_s, s_l) -> (b_s, s_l, n_l)
        pred_factor = ((1-pred_soft) ** self.alpha) if self.self_adjusting else 1
        if mask is not None:
            pred_soft = pred_soft * mask.unsqueeze(-1)
            target_one_hot = target_one_hot * mask.unsqueeze(-1)
        intersection = torch.sum(pred_factor * pred_soft * target_one_hot, 1) # (b_s,s_l,n_l)->(b_s,n_l)
        cardinality = torch.sum(pred_factor * pred_soft + target_one_hot, 1)  # (b_s,s_l,n_l)->(b_s,n_l)
        dice_score = 1. - 2. * (intersection + self.smooth) / (cardinality + self.smooth) * torch.tensor(self.weight)
        if self.reduction == "mean":
            return dice_score.mean()
        elif self.reduction == "sum":
            return dice_score.sum()
        elif self.reduction == "none" or self.reduction is None:
            return dice_score
        else:
            raise NotImplementedError(f"Reduction `{self.reduction}` is not supported.")
    def __str__(self):
        return f"Dice Loss smooth:{self.smooth}"

class DiceCRF(nn.Module):
    def __init__(
        self, 
        num_labels: int, 
        pad_idx: Optional[int] = None,
        reduction: Optional[str] = "mean",
    ) -> None:
        """
        :param num_labels: number of labels
        :param pad_idx: padding index. default None
        :return None
        """
        if num_labels < 1: raise ValueError("invalid number of labels: {0}".format(num_labels))
        super().__init__()
        self.num_labels = num_labels
        self.trans_matrix = nn.Parameter(torch.empty(num_labels, num_labels))
        self.start_trans = nn.Parameter(torch.empty(num_labels))
        self.end_trans = nn.Parameter(torch.empty(num_labels))
        self.reduction = reduction
        self._initialize_parameters(pad_idx)
    def forward(
        self, h: FloatTensor, labels: LongTensor, mask: ByteTensor,
    ) -> FloatTensor:
        """
        :param h: hidden matrix (batch_size, seq_len, num_labels)
        :param labels: answer labels of each sequence
                       in mini batch (batch_size, seq_len)
        :param mask: mask tensor of each sequence #attention_mask
                     in mini batch (batch_size, seq_len)
        :param reduction: Specifies  the reduction to apply to the output:
                ``none|sum|mean|token_mean``. ``none``: no reduction will be applied.
                ``sum``: the output will be summed over batches. ``mean``: the output will be
                averaged over batches. ``token_mean``: the output will be averaged over tokens
        :return: The log-likelihood (batch_size,) if reduction==none,  else ()
        """
        self._validate(h, labels=labels, mask=mask)
        if self.reduction not in ('none', 'sum', 'mean', 'token_mean'):
            raise ValueError(f'invalid reduction: {self.reduction}')
        if mask is None:
            mask = torch.ones_like(tags, dtype=torch.uint8)

        log_numerator = self._compute_numerator_log_likelihood(h, labels, mask) #score
        log_denominator = self._compute_denominator_log_likelihood(h, mask) #partition
        llh=log_numerator - log_denominator
        if self.reduction == 'none':
            return -llh
        if self.reduction == 'sum':
            return -llh.sum()
        if self.reduction == 'mean':
            return -llh.mean()
        assert self.reduction == 'token_mean'
        return -llh.sum() / mask.type_as(h).sum()

    def _validate(
            self,
            h: torch.Tensor,
            labels: Optional[LongTensor] = None,
            mask: Optional[ByteTensor] = None) -> None:
        if h.dim() != 3:
            raise ValueError(f'h must have dimension of 3, got {h.dim()}')
        if h.size(2) != self.num_labels:
            raise ValueError(
                f'expected last dimension of h is {self.num_labels}, '
                f'got {h.size(2)}')
        if labels is not None:
            if h.shape[:2] != labels.shape:
                raise ValueError(
                    'the first two dimensions of h and labels must match, '
                    f'got {tuple(h.shape[:2])} and {tuple(labels.shape)}')
        if mask is not None:
            if h.shape[:2] != mask.shape:
                raise ValueError(
                    'the first two dimensions of h and mask must match, '
                    f'got {tuple(h.shape[:2])} and {tuple(mask.shape)}')
    def decode(self, h: FloatTensor,
               mask: Optional[ByteTensor] = None) -> torch.Tensor:
        self._validate(h, mask=mask)

        if mask is None:
            mask = emissions.new_ones(emissions.shape[:2], dtype=torch.uint8)
        return self._viterbi_decode(h,mask)

    def _viterbi_decode(self, h: FloatTensor, mask: ByteTensor) -> torch.Tensor:
        """
        decode labels using viterbi algorithm
        :param h: hidden matrix (batch_size, seq_len, num_labels)
        :param mask: mask tensor of each sequence
                     in mini batch (batch_size, seq_len)
        :return: labels of each sequence in mini batch
        """
        assert h.dim() == 3 and mask.dim() == 2
        assert h.shape[:2] == mask.shape
        assert h.size(2) == self.num_labels
        assert mask[0].all()

        batch_size, seq_len, _ = h.shape
        # prepare the sequence lengths in each sequence
        seq_lens = mask.sum(dim=1)
        # In mini batch, prepare the score
        # from the start sequence to the first label
        score = [self.start_trans.data + h[:, 0]]
        path = []
        for t in range(1, seq_len):
            # extract the score of previous sequence
            # (batch_size, num_labels, 1)
            previous_score = score[t - 1].view(batch_size, -1, 1)
            # extract the score of hidden matrix of sequence
            # (batch_size, 1, num_labels)
            h_t = h[:, t].view(batch_size, 1, -1)
            # extract the score in transition
            # from label of t-1 sequence to label of sequence of t
            # self.trans_matrix has the score of the transition
            # from sequence A to sequence B
            # (batch_size, num_labels, num_labels)
            score_t = previous_score + self.trans_matrix + h_t
            # keep the maximum value
            # and point where maximum value of each sequence
            # (batch_size, num_labels)
            best_score, best_path = score_t.max(1)
            score.append(best_score)
            path.append(best_path)
        # predict labels of mini batch
        best_paths = [
            self._viterbi_compute_best_path(i, seq_lens, score, path)
            for i in range(batch_size)
        ]
        return torch.tensor(best_paths)

    def _viterbi_compute_best_path(
        self,
        batch_idx: int,
        seq_lens: torch.LongTensor,
        score: List[FloatTensor],
        path: List[torch.LongTensor],
    ) -> List[int]:
        """
        return labels using viterbi algorithm
        :param batch_idx: index of batch
        :param seq_lens: sequence lengths in mini batch (batch_size)
        :param score: transition scores of length max sequence size
                      in mini batch [(batch_size, num_labels)]
        :param path: transition paths of length max sequence size
                     in mini batch [(batch_size, num_labels)]
        :return: labels of batch_idx-th sequence
        """
        seq_end_idx = seq_lens[batch_idx] - 1
        # extract label of end sequence
        _, best_last_label = (score[seq_end_idx][batch_idx] + self.end_trans).max(0)
        best_labels = [int(best_last_label)]
        # predict labels from back using viterbi algorithm
        for p in reversed(path[:seq_end_idx]):
            best_last_label = p[batch_idx][best_labels[0]]
            best_labels.insert(0, int(best_last_label))
        return best_labels

    def _compute_denominator_log_likelihood(self, h: FloatTensor, mask: ByteTensor):
        """
        compute the denominator term for the log-likelihood
        compute the partition function in log-space using the forward-algorithm.
        :param h: hidden matrix (batch_size, seq_len, num_labels)
        :param mask: mask tensor of each sequence
                     in mini batch (batch_size, seq_len)
        :return: The score of denominator term for the log-likelihood
        """
        device = h.device
        batch_size, seq_len, _ = h.size()
        # (num_labels, num_labels) -> (1, num_labels, num_labels)
        trans = self.trans_matrix.unsqueeze(0)
        # add the score from beginning to each label
        # and the first score of each label
        score = self.start_trans + h[:, 0]
        # iterate through processing for the number of words in the mini batch
        for t in range(1, seq_len):
            # (batch_size, self.num_labels, 1)
            before_score = score.unsqueeze(2)
            # prepare t-th mask of sequences in each sequence
            # (batch_size, 1)
            mask_t = mask[:, t].unsqueeze(1)
            mask_t = mask_t.to(device)
            # prepare the transition probability of the t-th sequence label
            # in each sequence
            # (batch_size, 1, num_labels)
            h_t = h[:, t].unsqueeze(1)
            # calculate t-th scores in each sequence
            # (batch_size, num_labels)
            score_t = before_score + h_t + trans
            score_t = torch.logsumexp(score_t, 1)
            # update scores
            # (batch_size, num_labels)
            score = torch.where(mask_t, score_t, score)
        # add the end score of each label
        score += self.end_trans
        # return the log likely food of all data in mini batch
        return torch.logsumexp(score, 1)

    def _compute_numerator_log_likelihood(
        self, h: FloatTensor, y: LongTensor, mask: ByteTensor
    ) -> FloatTensor:
        """
        compute the numerator term for the log-likelihood
        :param h: hidden matrix (batch_size, seq_len, num_labels)
        :param y: answer labels of each sequence
                  in mini batch (batch_size, seq_len)
        :param mask: mask tensor of each sequence
                     in mini batch (batch_size, seq_len)
        :return: The score of numerator term for the log-likelihood
        """
        batch_size, seq_len, _ = h.size()
        h_unsqueezed = h.unsqueeze(-1)
        trans = self.trans_matrix.unsqueeze(-1)
        arange_b = torch.arange(batch_size)
        # extract first vector of sequences in mini batch
        last_mask_index = mask.sum(1) - 1
        calc_range = seq_len - 1 #should calc_range be last_mask_index? No since parallel faster
        # calc_range = last_mask_index
        score = self.start_trans[y[:, 0]] + sum(
            [self._calc_trans_score_for_num_llh(
                h_unsqueezed, y, trans, mask, t, arange_b
            ) for t in range(calc_range)])
        # extract end label number of each sequence in mini batch
        # (batch_size)
        last_labels = y[arange_b,last_mask_index]
        each_last_score = h[arange_b, -1, last_labels].squeeze(-1) * mask[:, -1]
        # Add the score of the sequences of the maximum length in mini batch
        # Add the scores from the last tag of each sequence to EOS
        score += each_last_score + self.end_trans[last_labels]
        return score
    def _calc_trans_score_for_num_llh(
        self,
        h: FloatTensor,
        y: LongTensor,
        trans: FloatTensor,
        mask: ByteTensor,
        t: int,
        arange_b: FloatTensor,
    ) -> torch.Tensor:
        """
        calculate transition score for computing numerator llh
        :param h: hidden matrix (batch_size, seq_len, num_labels)
        :param y: answer labels of each sequence
                  in mini batch (batch_size, seq_len)
        :param trans: transition score
        :param mask: mask tensor of each sequence
                     in mini batch (batch_size, seq_len)
        :paramt t: index of hidden, transition, and mask matrixex
        :param arange_b: this param is seted torch.arange(batch_size)
        :param batch_size: batch size of this calculation
        """
        device = h.device
        mask_t = mask[:, t]
        mask_t = mask_t.to(device)
        mask_t1 = mask[:, t + 1]
        mask_t1 = mask_t1.to(device)
        # extract the score of t label
        # (batch_size)
        h_t = h[arange_b, t, y[:, t]].squeeze(1)  ##Changed from t to t
        # extract the transition score from t-th label to t+1 label
        # (batch_size)
        trans_t = trans[y[:, t], y[:, t + 1]].squeeze(1)
        # add the score of t+1 and the transition score
        # (batch_size)
        return h_t * mask_t + trans_t * mask_t1
    def _initialize_parameters(self, pad_idx: Optional[int]) -> None:
        """
        initialize transition parameters
        :param: pad_idx: if not None, additional initialize
        :return: None
        """
        nn.init.uniform_(self.trans_matrix, -0.1, 0.1)
        nn.init.uniform_(self.start_trans, -0.1, 0.1)
        nn.init.uniform_(self.end_trans, -0.1, 0.1)
        punct=torch.arange(1,self.trans_matrix.shape[0])
        # inaccurate since consecutive punctuation is possible ("Hello! Bye!")
        # with torch.no_grad():
        #   self.trans_matrix[punct,punct]=-100
        if pad_idx is not None:
            self.start_trans[pad_idx] = -10000.0
            self.trans_matrix[pad_idx, :] = -10000.0
            self.trans_matrix[:, pad_idx] = -10000.0
            self.trans_matrix[pad_idx, pad_idx] = 0.0

class CrossEntropyLoss(nn.Module):
    """
    This criterion (`CrossEntropyLoss`) combines `LogSoftMax` and `NLLLoss` in one single class.
    
    NOTE: Computes per-element losses for a mini-batch (instead of the average loss over the entire mini-batch).
    """
    
    def __init__(self, weight: Optional[torch.Tensor] = None, ignore_index: int = -100, reduction: Optional[str] = 'mean') -> None:
        super().__init__()
        self.weight=weight
        self.ignore_index = ignore_index
        self.reduction=reduction

    def forward(self, input: FloatTensor, target: LongTensor, mask: ByteTensor) -> torch.Tensor:
        assert self.weight is None or isinstance(self.weight, torch.Tensor)
        return F.cross_entropy(input[mask].view(-1,10).float(), target[mask].long(), weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction)

In [236]:
#b_s = 4, s_l = 128, h_d = 768
bert=transformers.BertModel.from_pretrained(config.base_model_path)
input_ids, attention_mask, labels = next(iter(train_dataloader)).values() # (batch_size, seq_len) * 3
bo=bert(input_ids, attention_mask) # => last_hidden_state (b_s, s_l, h_d 768), pooler_output (b_s, h_d)
dropout = nn.Dropout(config.hidden_dropout_prob) # (b_s, s_l, h_d)
sequence_output=dropout(bo[0])
fcl = nn.Linear(config.embedding_dim, config.num_labels)
fcl_output=fcl(sequence_output) #(b_s, s_l, h_d) -> (b_s, s_l, num_labels)
dice_loss=DiceLoss(self_adjusting=True,alpha=1)(fcl_output,labels,attention_mask)
## bert(next)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'd

In [262]:
class BertCRFModel(BertPreTrainedModel):
    def __init__(self,config):
        super().__init__(config)
        self.num_labels=config.num_labels
        self.embedding_dim=config.embedding_dim
        self.use_crf = config.use_crf
        self.use_dice = config.use_dice
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.bert = transformers.BertModel.from_pretrained(config.base_model_path)
        self.fcl = nn.Linear(self.embedding_dim, self.num_labels)
        if self.use_crf:
            self.loss=DiceCRF(self.num_labels,reduction=config.reduction)
        elif self.use_dice:
            self.loss=DiceLoss(self_adjusting=config.self_adjusting,alpha=config.alpha, num_classes=self.num_labels, reduction=config.reduction)
        else:
            self.loss=CrossEntropyLoss(reduction=config.reduction)
    def forward(self, input_ids:FloatTensor, attn_masks:ByteTensor, labels:Optional[LongTensor]=None):
        """
        :param input_ids: tokenids (batch_size,seq_len)
        :param attention_mask: attention_mask, null->0 (batch_size,seq_len)
        :param labels: true labels, optional (batch_size,seq_len)
        :return None
        """
        o1 = self.bert(input_ids,attn_masks)[0] #(batch_size,seq_len), (batch_size,seq_len)-> last_hidden_state (batch_size,seq_len,hidden_size)
        sequence_output = self.dropout(o1) #(batch_size,seq_len,hidden_size)
        punct = self.fcl(sequence_output) #(batch_size,seq_len,hidden_size) -> (batch_size,seq_len,num_labels)
        if labels is not None:
            return self.loss(punct, labels, attn_masks)
        else:
            prediction = F.one_hot(self.loss.decode(punct, attn_masks).flatten(),10) if self.use_crf else punct.view(-1,10)
            #(batch_size*seq_len,10)
            return prediction
        #loss = (loss_tag + loss_pos) / 2
model=BertCRFModel(config)
for i,param in enumerate(model.bert.parameters()):
    param.requires_grad = False
# model.to(device)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'd

In [261]:
# model(input_ids, attention_mask, labels)
# model.train()
# model(input_ids, attention_mask,labels)
# fcl_output.view(-1,10).shape
# labels=DiceCRF(10).decode(fcl_output,attention_mask)

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [ ]:
# train_dataloader, dev_dataloader
from ignite.utils import setup_logger, convert_tensor
from ignite.engine.engine import Engine
from ignite.metrics import Fbeta, ConfusionMatrix, DiceCoefficient, Precision, Recall
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Metric
from ignite.contrib.handlers.tensorboard_logger import *


model.train()
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.001},
    {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     "weight_decay": 0.0}
     ]
base_opt = AdamW(optimizer_parameters, lr=config.freeze_lr)
optimizer = SWA(base_opt)

def _prepare_batch(
    batch: Sequence[torch.Tensor], device: Optional[Union[str, torch.device]] = None, non_blocking: bool = False
):
    """Prepare batch for training: pass to a device with options."""
    x, y, z = batch.values()
    return (convert_tensor(x, device=device, non_blocking=non_blocking),
            convert_tensor(y, device=device, non_blocking=non_blocking),
            convert_tensor(y, device=device, non_blocking=non_blocking))

def create_supervised_trainer(
    model: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    loss_fn: Optional[Union[Callable, torch.nn.Module]] = None,
    device: Optional[Union[str, torch.device]] = None,
    non_blocking: bool = False,
    prepare_batch: Callable = _prepare_batch,
    output_transform: Callable = lambda x, y, loss: loss.item(),
    deterministic: bool = False) -> Engine:

    device_type = device.type if isinstance(device, torch.device) else device
    on_tpu = "xla" in device_type if device_type is not None else False

    if on_tpu and not idist.has_xla_support:
        raise RuntimeError("In order to run on TPU, please install PyTorch XLA")

    def _update(engine: Engine, batch: Sequence[torch.Tensor]) -> Union[Any, Tuple[torch.Tensor]]:
        model.train()
        optimizer.zero_grad()
        input_ids, attention_mask, labels = prepare_batch(batch, device=device, non_blocking=non_blocking)
        loss = model(input_ids, attention_mask, labels)
        loss.backward()

        if on_tpu:
            xm.optimizer_step(optimizer, barrier=True)
        else:
            optimizer.step()

        return output_transform(input_ids, attention_mask, loss)

    trainer = Engine(_update) if not deterministic else DeterministicEngine(_update)

    return trainer

def create_supervised_evaluator(
    model: torch.nn.Module,
    metrics: Optional[Dict[str, Metric]] = None,
    device: Optional[Union[str, torch.device]] = None,
    non_blocking: bool = False,
    prepare_batch: Callable = _prepare_batch,
    output_transform: Callable = lambda x, y, y_pred: (y_pred, y)) -> Engine:
    metrics = metrics or {}

    def _inference(engine: Engine, batch: Sequence[torch.Tensor]) -> Union[Any, Tuple[torch.Tensor]]:
        model.eval()
        with torch.no_grad():
            input_ids, attention_mask, labels = prepare_batch(batch, device=device, non_blocking=non_blocking)
            y_pred = model(input_ids, attention_mask)
            return output_transform(x, y, y_pred)

    evaluator = Engine(_inference)

    for name, metric in metrics.items():
        metric.attach(evaluator, name)

    return evaluator

trainer = create_supervised_trainer(model, optimizer, device=device)
trainer.logger = setup_logger("Trainer")
metrics = {"F1": Fbeta(1), "dice": DiceCoefficient(ConfusionMatrix(num_classes=10)),"p": Precision(), 'r': Recall()}
train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
train_evaluator.logger = setup_logger("Train Evaluator")
dev_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
dev_evaluator.logger = setup_logger("Dev Evaluator")

@trainer.on(Events.EPOCH_COMPLETED)
def compute_metrics(engine):
    train_evaluator.run(train_dataloader)
    dev_evaluator.run(dev_dataloader)

tb_logger = TensorboardLogger(log_dir=config.log_dir)
tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batchloss": loss},
    metric_names="all",
)
for tag, evaluator in [("training", train_evaluator), ("validation", dev_evaluator)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names=["F1", "dice"],
        global_step_transform=global_step_from_engine(trainer),
    )

def score_function(engine):
    return engine.state.metrics["F1"]

model_checkpoint = ModelCheckpoint(
    config.log_dir,
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="validation_accuracy",
    global_step_transform=global_step_from_engine(trainer),
)

dev_evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})

trainer.run(train_dataloader, max_epochs=config.freeze_epochs)
tb_logger.close()

2021-01-08 04:25:47,378 Trainer INFO: Engine run starting with max_epochs=20.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:348: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py:132: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
  allow_unreachable=True)  # allow_unreachable flag


In [267]:
!pip install pytorch-ignite
# !pip uninstall ignite

#exploration dice and crf

In [ ]:
# dice_loss(fcl_output,labels,attention_mask)
#b_s,s_l,n_l
# pred_soft=torch.softmax(fcl_output,-1)#,torch.softmax(fcl_output[0,0,:],-1) #apply softmax to each token
# target_one_hot=F.one_hot(labels,num_classes=config.num_labels)#,labels[0,:5] (b_s, s_l) -> (b_s, s_l, n_l)
# pred_factor=((1-pred_soft) ** config.alpha) if config.self_adjusting else 1
# pred_prod=pred_factor*pred_soft*target_one_hot
# sum(pred_prod,0).shape
# smooth = 1e-8
# intersection=torch.sum(pred_prod,1)
intersection.shape,pred_prod.shape
# cardinality =torch.sum(pred_factor*pred_soft + target_one_hot, 1)
# dice_score=1-2*(intersection+smooth)/(cardinality+smooth)
# dice_score[0,:]
# weight=[0,0,0,0,0,0,0,1,0,0]
# (dice_score[:2,:]*torch.tensor(torch.tensor(weight))).shape
### torch.gather(pred_soft[0,:5],-1,index=labels[0,:5].unsqueeze(-1)) #returns the probability for the most likely example is this really needed?
# target_one_hot.shape
# labels.unsqueeze(2).shape,labels.shape
# pred_soft[0,:5].shape

(torch.Size([4, 10]), torch.Size([4, 128, 10]))

In [ ]:
# crf_score=DiceCRF(10)(fcl_output,labels,attention_mask,'mean')
# crf_score
DiceCRF(10).decode(fcl_output,attention_mask)

In [98]:
def validate(
        h: torch.Tensor,
        labels: Optional[torch.LongTensor] = None,
        mask: Optional[torch.ByteTensor] = None) -> None:
    if h.dim() != 3:
        raise ValueError(f'h must have dimension of 3, got {h.dim()}')
    if h.size(2) != config.num_labels:
        raise ValueError(
            f'expected last dimension of h is {config.num_labels}, '
            f'got {h.size(2)}')
    if labels is not None:
        if h.shape[:2] != labels.shape:
            raise ValueError(
                'the first two dimensions of h and labels must match, '
                f'got {tuple(h.shape[:2])} and {tuple(labels.shape)}')
    if mask is not None:
        if h.shape[:2] != mask.shape:
            raise ValueError(
                'the first two dimensions of h and mask must match, '
                f'got {tuple(h.shape[:2])} and {tuple(mask.shape)}')
# validate(fcl_output,labels) #why must x be the fcl output?

# num_labels=10
# trans_matrix = nn.Parameter(torch.empty(num_labels, num_labels))
# start_trans = nn.Parameter(torch.empty(num_labels))
# end_trans = nn.Parameter(torch.empty(num_labels))
# def _initialize_parameters(pad_idx: Optional[int]) -> None:
#       """
#       initialize transition parameters
#       :param: pad_idx: if not None, additional initialize
#       :return: None
#       """
#       nn.init.uniform_(trans_matrix, -0.1, 0.1)
#       nn.init.uniform_(start_trans, -0.1, 0.1)
#       nn.init.uniform_(end_trans, -0.1, 0.1)
#       punct=torch.arange(1,trans_matrix.shape[0])
#       # inaccurate since consecutive punctuation is possible ("Hello! Bye!")
#       # with torch.no_grad():
#       #   self.trans_matrix[punct,punct]=-100
#       if pad_idx is not None:
#           start_trans[pad_idx] = -10000.0
#           trans_matrix[pad_idx, :] = -10000.0
#           trans_matrix[:, pad_idx] = -10000.0
#           trans_matrix[pad_idx, pad_idx] = 0.0
# _initialize_parameters(None)

def _compute_numerator_log_likelihood(
    h: FloatTensor, y: LongTensor, mask: ByteTensor
) -> FloatTensor:
    """
    compute the numerator term for the log-likelihood
    :param h: hidden matrix (batch_size, seq_len, num_labels)
    :param y: answer labels of each sequence
              in mini batch (batch_size, seq_len)
    :param mask: mask tensor of each sequence
                  in mini batch (batch_size, seq_len)
    :return: The score of numerator term for the log-likelihood
    """
    batch_size, seq_len, _ = h.size()
    h_unsqueezed = h.unsqueeze(-1)
    trans = trans_matrix.unsqueeze(-1)
    arange_b = torch.arange(batch_size)
    # extract first vector of sequences in mini batch
    last_mask_index = mask.sum(1) - 1
    calc_range = seq_len - 1 #should calc_range be last_mask_index? No since parallel faster
    # calc_range = last_mask_index
    score = start_trans[y[:, 0]] + sum(
        [_calc_trans_score_for_num_llh(
            h_unsqueezed, y, trans, mask, t, arange_b
        ) for t in range(calc_range)])
    # extract end label number of each sequence in mini batch
    # (batch_size)
    last_labels = y[arange_b,last_mask_index]
    each_last_score = h[arange_b, -1, last_labels].squeeze(-1) * mask[:, -1]
    # Add the score of the sequences of the maximum length in mini batch
    # Add the scores from the last tag of each sequence to EOS
    score += each_last_score + end_trans[last_labels]
    return score
def _calc_trans_score_for_num_llh(
    h: FloatTensor,
    y: LongTensor,
    trans: FloatTensor,
    mask: ByteTensor,
    t: int,
    arange_b: FloatTensor,
) -> torch.Tensor:
    """
    calculate transition score for computing numerator llh
    :param h: hidden matrix (batch_size, seq_len, num_labels)
    :param y: answer labels of each sequence
              in mini batch (batch_size, seq_len)
    :param trans: transition score
    :param mask: mask tensor of each sequence
                  in mini batch (batch_size, seq_len)
    :paramt t: index of hidden, transition, and mask matrixex
    :param arange_b: this param is seted torch.arange(batch_size)
    :param batch_size: batch size of this calculation
    """
    device = h.device
    mask_t = mask[:, t]
    mask_t = mask_t.to(device)
    mask_t1 = mask[:, t + 1]
    mask_t1 = mask_t1.to(device)
    # extract the score of t label
    # (batch_size)
    h_t = h[arange_b, t, y[:, t]].squeeze(1)  ##Changed from t to t
    # extract the transition score from t-th label to t+1 label
    # (batch_size)
    trans_t = trans[y[:, t], y[:, t + 1]].squeeze(1)
    # add the score of t+1 and the transition score
    # (batch_size)
    return h_t * mask_t + trans_t * mask_t1

# numerator_log_likelihood=_compute_numerator_log_likelihood(fcl_output,labels,attention_mask.byte())
# numerator_log_likelihood
def _compute_denominator_log_likelihood(h: FloatTensor, mask: ByteTensor):
        device = h.device
        batch_size, seq_len, _ = h.size()
        # (num_labels, num_labels) -> (1, num_labels, num_labels)
        trans = trans_matrix.unsqueeze(0)
        # add the score from beginning to each label
        # and the first score of each label
        score = start_trans + h[:, 0]
        # iterate through processing for the number of words in the mini batch
        for t in range(1, seq_len):
            before_score = score.unsqueeze(2) # (batch_size, num_labels) -> (batch_size, num_labels, 1)
            # prepare t-th mask of sequences in each sequence
            # (batch_size, 1)
            mask_t = mask[:, t].unsqueeze(1)
            mask_t = mask_t.to(device)
            # prepare the transition probability of the t-th sequence label
            # in each sequence
            # (batch_size, 1, num_labels)
            h_t = h[:, t].unsqueeze(1)
            # calculate t-th scores in each sequence
            # (batch_size, num_labels)
            score_t = before_score + h_t + trans
            score_t = torch.logsumexp(score_t, 1)
            # update scores
            # (batch_size, num_labels)
            score = torch.where(mask_t, score_t, score)
        print(score)
        # add the end score of each label
        score += end_trans
        print(score)
        # return the log likelihood of all data in mini batch
        return torch.logsumexp(score, 1)
# denominator_log_likelihood=_compute_denominator_log_likelihood(fcl_output,attention_mask.byte())
# denominator_log_likelihood

# llh=numerator_log_likelihood - denominator_log_likelihood
numerator_log_likelihood

tensor([46.9585, 50.8604, 49.0791, 51.3038], grad_fn=<AddBackward0>)

#-

In [ ]:
### engine.py
# from tqdm import tqdm
def train_step(trainer,batch):
    model.train()
    optimizer.zero_grad()
    batch=[_data.to(device) for _data in batch]
    _, loss = model(batch)
    loss.backward()
    optimizer.step()
    scheduler.step()
    return loss.item()

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        if torch.cuda.is_available(): data=[_data.to(device) for _data in data]
        optimizer.zero_grad()
        _, loss = model(data)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)


def eval_fn(data_loader, model, device):
    model.eval()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        if torch.cuda.is_available(): data=[_data.to(device) for _data in data]
        punct, loss = model(data)
        final_loss += loss.item()
    return final_loss / len(data_loader)


In [ ]:
model = BertCRFModel(num_punct=10, embedding_dim=config.EMBEDDING_DIM, hidden_dim=config.HIDDEN_DIM, use_crf=config.USE_CRF)
for i,param in enumerate(model.bert.parameters()):
    param.requires_grad = False
model.to(device)
optimizer = AdamW(optimizer_parameters, lr=config.FREEZE_LEARNING_RATE)
num_train_steps = train_dataset.tensors[0].size()[0] / config.TRAIN_BATCH_SIZE * config.UNFREEZE_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)
# optimizer = SWA(base_opt)

trainer = Engine(train_step)
val_metrics = {
    "precision": Precision(),
    "recall": Recall(),
#     "Dice": DiceCoefficient(cm=),
    "F1": Fbeta(1),
}
evaluator = create_supervised_evaluator(model, metrics=val_metrics)
def log_metrics(engine, title):
    print("Epoch: {} - {} accuracy: {:.2f}"
           .format(trainer.state.epoch, title, engine.state.metrics["acc"]))

@trainer.on(Events.EPOCH_COMPLETED)
def evaluate(trainer):
    with evaluator.add_event_handler(Events.COMPLETED, log_metrics, "train"):
        evaluator.run(train_dataloader)

    with evaluator.add_event_handler(Events.COMPLETED, log_metrics, "dev"):
        evaluator.run(dev_dataloader)

trainer.run(train_dataloader, max_epochs=100)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'd

Current run is terminating due to exception: too many values to unpack (expected 2).
Engine run is terminating due to exception: too many values to unpack (expected 2).
Engine run is terminating due to exception: too many values to unpack (expected 2).


ValueError: too many values to unpack (expected 2)

In [ ]:
torch.utils.data.DataLoader??